## Searching original tweets

Let's look for the original tweets

In [1]:
%load_ext autoreload
%autoreload 2
from mongoengine import connect
from tweepyrate import create_apps

client = connect("hatespeech")
apps = create_apps("../config/my_apps.json")
db = client["hatespeech"]

7 apps created


In [2]:
db.tweet.find({}).distinct('query')

[None,
 'facha',
 'facho',
 'fascista',
 'machirulo',
 'machista',
 'misogino',
 'misógino',
 'racista']

In [3]:
from hate_collector.models import Tweet

replies = Tweet.objects(
    in_reply_to_status_id__exists= True, 
    in_reply_to_status_id__ne= None,
).limit(10)


In [4]:
from mongoengine import DoesNotExist, NotUniqueError
from tweepy import TweepError


count = 0
for i, tweet in enumerate(replies):
    app = apps[i % len(apps)]
    print("="*80)
        
    try:
        
        status = app.get_status(tweet["in_reply_to_status_id"], tweet_mode="extended")
        print("CON RESPUESTA\n")
        
        upstream_tweet = Tweet(**status._json)
        upstream_tweet.save()
        
        print(f"{upstream_tweet.text}")
        count += 1
    except TweepError:
        print("SIN RESPUESTA\n")
    print(" ---- ", tweet["text"])
    

CON RESPUESTA

@asaberquedicen @davidsantosvlog Tiene pinta de español, quemado al sol.
 ----  @elenavpz1 @asaberquedicen @davidsantosvlog Racista.
CON RESPUESTA

El candidato de Solidaridad Nacional es muy criticado en redes tras su accionar luego de un debate y fue acusado de racista por el mismo Arbizu.
https://t.co/QyO8Fq844D
 ----  @Caretas No es necesario, acusarlo!!!! Se pinta de cuerpo entero, es racista y punto. Ahora que asuma su posición, que saque el jaboncito y lo use, a ver si se puede "bañar" y quedar limpiecito.
CON RESPUESTA

@P4P1T0_M0N3Y @1991carles @oscarsh98 @kikosaintchez @vox_es Ojo, me parece bien que se eche al inmigrante que cometa crimenes, es de lógica. Pero creo que se generaliza mucho y mal. Se hace mucho incapié en la nacionalidad de los criminales y eso solo incrementa los prejuicios.
 ----  @euricius @P4P1T0_M0N3Y @oscarsh98 @kikosaintchez @vox_es Pues ya está, estamos de acuerdo.

Puede que haya algunos que generalicen, claro que sí. Pero por otro lado